## Introduction

Traffic sign recognition is a challenging, real-world problem relevant for AI based transportation systems. Traffic signs show a wide range of variations between classes in terms of color, shape, and the presence of pictograms or text. However, there exist subsets of
classes (e.g., speed limit signs) that are very similar to each other. Further, the classifier
has to be robust against large variations in visual appearances due to changes in illumination, partial
occlusions, rotations, weather conditions etc. Using a comprehensive traffic sign detection dataset, here we will perform classification of traffic signs, train and evaluate the different models and compare to the performance of MLPs.

![img](https://paperswithcode.com/media/datasets/GTSRB-0000000633-9ce3c5f6_Dki5Rsf.jpg)

## Dataset

The data for this mini-project is from the German Traffic Sign Detection Benchmark [GTSDB](https://benchmark.ini.rub.de/gtsdb_dataset.html). This archive contains the training set used during the IJCNN 2013 competition.

The German Traffic Sign Detection Benchmark is a single-image detection assessment for researchers with interest in the field of computer vision, pattern recognition and image-based driver assistance. It is introduced on the IEEE International Joint Conference on Neural Networks 2013.

It features ...

* The main archive FullIJCNN2013.zip includes the images (1360 x 800 pixels) in PPM format, the image sections containing only the traffic signs
* A file in CSV format with the ground truth
* A ReadMe.txt with more details.

Note that we will be using the images inside the image sections subfolders, containing only the traffic signs.

## Problem Statement

To build and improve upon a machine learning model for the classification of images and achieve a high accuracy final model.

In [ ]:
#@title Download the data
!wget -qq https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip
!unzip -qq FullIJCNN2013.zip

### Import Required packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from skimage.io import imread, imshow
from sklearn import preprocessing
import os, glob
from PIL import Image
from sklearn.model_selection import GridSearchCV
# Keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [ ]:
images_data = glob.glob("/content/FullIJCNN2013/*/*.ppm")
len(images_data), images_data[0]

In [ ]:
features, labels = [], []
for i in images_data:
    try:
        img = Image.open(i)
        img = img.resize((30,30))#.reshape(30*30*3)
        labels.append(int(i.split("/")[3]))
        features.append(np.array(img))
    except:
        pass

In [ ]:
features[0].shape

In [ ]:
features1 = np.array([i.reshape(-1) for i in features])
features1.shape

In [ ]:
n_classes = len(set(labels))
n_classes

### Data Exploration and Preprocessing

#### Plot the sample image of each class

In [ ]:
plt.imshow(imread('/content/FullIJCNN2013/01/00002.ppm'))
imread('/content/FullIJCNN2013/01/00002.ppm').shape

In [ ]:
targets = np.array(labels)
plt.figure(figsize=(16, 16))
for c in range(n_classes):
    i = np.random.choice(np.where(targets == c)[0])
    plt.subplot(8, 8, c+1)
    plt.axis('off')
    plt.title('class: {}'.format(c))
    plt.imshow(features[i])

#### Plot the distribution of Classes

In [ ]:
# Visulization of the histogram
images_per_class, bins, _ = plt.hist(labels,bins=range(n_classes))
plt.xticks(range(n_classes))
plt.title('histogram')

#### Normalize the features

For most image data, the pixel values are integers with values between 0 and 255.

Neural networks process inputs using small weight values, and inputs with large integer values can disrupt or slow down the learning process. As such it is good practice to normalize the pixel values.

In [ ]:
features_norm = preprocessing.normalize(features1, norm='l2')
features_norm.shape

### Train the MLP classifier on features

* Split the data into train and test

* Train the MLP classifier with different parameters

* Get the accuracy score and performance metrics

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(np.array(features_norm), np.array(labels), test_size=0.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

### Tune the hyper-parameters

In [ ]:
mlp = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(450,225,100,50), (100,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, Y_train)

In [ ]:
# Best parameter set
print('Best parameters found:\n', clf.best_params_)

In [ ]:
mlp = MLPClassifier(activation='tanh',momentum=0.99,alpha=0.0001,max_iter=1000,
                    hidden_layer_sizes=(450,225,100, 50),learning_rate='adaptive',solver='adam')
mlp.fit(X_train, Y_train)
mlp.score(X_test, Y_test)

#### Classification report

In [ ]:
pred_test = mlp.predict(X_test)
from sklearn import metrics
print(metrics.classification_report(Y_test, pred_test))

In [ ]:
metrics.confusion_matrix(Y_test, pred_test)

In [ ]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w,3)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]]
    true_name = target_names[y_test[i]]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(pred_test, Y_test, Y_test, i)
                     for i in range(pred_test.shape[0])]

plot_gallery(X_test, prediction_titles, 30, 30)

#### Use RandomSearchCV to search the hyper parameters

In [ ]:
import scipy.stats
from sklearn.model_selection import RandomizedSearchCV

mlp = MLPClassifier(early_stopping=True)
randomCV = RandomizedSearchCV(mlp,parameter_space)
search = randomCV.fit(X_train, Y_train)
search.best_params_

#### Try the different algorithms and compare the results with MLP classifier

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [ ]:
!pip install  xgboost
from xgboost import XGBClassifier

In [ ]:
# Decision Tree
dt_model_team = tree.DecisionTreeClassifier(random_state=42)
dt_model_team = dt_model_team.fit(X_train, Y_train)
dt_model_team.score(X_test, Y_test), dt_model_team.score(X_train, Y_train)

In [ ]:
# Random Forest
rf_model_team = RandomForestClassifier(max_depth=10, n_estimators=250,criterion='entropy',random_state=42)
rf_model_team.fit(X_train, Y_train)
rf_model_team.score(X_test, Y_test), rf_model_team.score(X_train, Y_train)

In [ ]:
#Linear SVC
Lsvm = LinearSVC(random_state=0, tol=1e-5)
Lsvm.fit(X_train, Y_train)
Lsvm.score(X_test, Y_test)

In [ ]:
svm_clf = SVC(kernel="rbf",C=1.0)
svm_clf.fit(X_train, Y_train)
svm_clf.score(X_test, Y_test)

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, Y_train)
xgb.score(X_test, Y_test)

### Implement simple Neural Networks using keras

* Define the keras model and initialize the layers
  - Ensure the input layer has the right number of input features. This can be specified when creating the first layer with the input_dim argument.
* Compile the model
  - Specify the loss function (to evaluate a set of weights), the optimizer (is used to search through different weights for the network) and any optional metrics to collect and report during training.
* Fit and Evaluate the model
  - Fit the data by specifying epochs and evaluate the model

In [ ]:
print(tf.__version__) # 1.12.0

In [ ]:
# Step 1 - Build the architecture
# Model a simple 3-layer neural network
nn_model = keras.Sequential([
    keras.layers.Flatten(input_shape=[2700]),
    keras.layers.Dense(1350, activation=tf.nn.relu),
    keras.layers.Dense(675, activation=tf.nn.relu),
    keras.layers.Dense(43, activation=tf.nn.softmax)
])
nn_model.summary()

In [ ]:
np.array(X_train).shape, Y_train.shape, len(set(labels))

In [ ]:
# Step 2 - Compile the model
nn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
nn_model.fit(X_train, Y_train, epochs=20)

In [ ]:
nn_model.evaluate(X_test, Y_test)

#### Try the same parameters used for MLP Classifier and build the keras model

In [ ]:
nn_model = keras.Sequential([
    keras.layers.Flatten(input_shape=[2700]),
    keras.layers.Dense(450, activation=tf.nn.tanh),
    keras.layers.Dense(225, activation=tf.nn.tanh),
    keras.layers.Dense(100, activation=tf.nn.tanh),
    keras.layers.Dense(50, activation=tf.nn.softmax)
])
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, Y_train, epochs=20)

In [ ]:
nn_model.evaluate(X_test, Y_test)

#### Experiment using Dropout, Regularization and Batch Normalization

In [ ]:
# With Regularization and DropOut and BatchNormalization
nn_model = keras.Sequential([
    keras.layers.Flatten(input_shape=[2700]),
    keras.layers.Dense(1350, activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(675, activation=tf.nn.relu),
    Dropout(rate=0.1),
    BatchNormalization(),
    keras.layers.Dense(43, activation=tf.nn.softmax)
])

nn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
nn_model.fit(X_train, Y_train, epochs=20)

In [ ]:
nn_model.evaluate(X_test, Y_test)

### Report Analysis

* According to the confusion matrix, for which sign were the maximum misclassifications observed? Comment on the misclassification, owing to similar appearing traffic signs, if any.
* Comment on the performance of the MLP Classifier
* Discuss the optimal number of layers, activation functions, optimizers etc. that yielded the best accuracy
* Report on training time vs convergence

In [ ]:
predict_test = nn_model.predict(X_test)
predict_test = np.argmax(predict_test,axis=1)
predict_test

In [ ]:
metrics.confusion_matrix(Y_test, predict_test)

In [ ]:
print(metrics.classification_report(Y_test, predict_test))